# Equation of state (EOS)

For the source code, see [eos](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.eos.eos_workgraph).

## Example: Silicon

### Prepare the inputs and submit the workflow



In [1]:
from ase.build import bulk
from ase import Atoms
from aiida import load_profile
from workgraph_collections.ase.espresso.eos import EosWorkGraph

load_profile()


# create input structure
atoms = bulk("Si")

metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"Si": "Si.pbe-nl-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
input_data = {
    "control": {"calculation": "scf"},
    "system": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold",},
}
#------------------------- Set the inputs -------------------------
wg = EosWorkGraph.build(atoms=atoms,
                   computer="localhost",
                   scales=[0.95, 0.98, 1.0, 1.02, 1.05],
                   command="mpirun -np 2 pw.x",
                   pseudopotentials=pseudopotentials,
                   pseudo_dir=pseudo_dir,
                   input_data=input_data,
                   kpts=[4, 4, 4],
                   metadata=metadata)
wg.to_html()

Run the calculation


In [2]:
wg.run()


09/12/2025 11:41:14 PM <1838528> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [187534|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[187534|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
09/12/2025 11:41:15 PM <1838528> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [187534|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 187536
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[187534|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 187536
09/12/2025 11:41:20 PM <1838528> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [187534|WorkGraphEngine|update_task_state]: Task: pw_calculator, type: PYTHONJOB, finished.
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[187534|WorkGraphEngine|update_task_state]: Task: pw_calculator, type: PYTHONJOB, fini

{'result': <Dict: uuid: 09e62649-4d48-4668-88e4-c0c59dd43da1 (pk: 187644)>}

Print the results

In [3]:
#------------------------- Print the output -------------------------
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**wg.outputs.result.value))



Result: 
B: 90.247955134586
v0: 41.302487642021
e0: -277.89706525605
v0: 41.302487642021
